# Streaming with Ollama & LangGraph

LLM 응답을 **실시간 스트리밍**으로 받는 방법을 알아봅니다.

## 스트리밍 방식

| 방식 | 설명 | 용도 |
|------|------|------|
| `astream` | 토큰 단위 스트리밍 | 기본 텍스트 스트리밍 |
| `astream_events` | 이벤트 단위 스트리밍 | 도구 호출, RAG 등 상세 추적 |

## 사전 요구사항
- Ollama 서버 실행 중
- `gemma3:12b` 모델 설치

## 1. 환경 설정

In [ ]:
import os
import asyncio
from dotenv import load_dotenv

load_dotenv()

OLLAMA_URL = os.getenv("OLLAMA_URL", "http://localhost:11434")
MODEL_NAME = "gemma3:12b"

print(f"Ollama URL: {OLLAMA_URL}")
print(f"Model: {MODEL_NAME}")

In [ ]:
from langchain_ollama import ChatOllama

llm = ChatOllama(
    model=MODEL_NAME,
    base_url=OLLAMA_URL,
    temperature=0.7,
)

print(f"{MODEL_NAME} 모델 초기화 완료!")

## 2. 기본 스트리밍 (astream)

토큰 단위로 응답을 받습니다.

In [ ]:
# 동기 스트리밍
print("[동기 스트리밍]\n")

for chunk in llm.stream("Python의 장점 3가지를 간단히 설명해줘"):
    print(chunk.content, end="", flush=True)

print("\n\n[완료]")

In [ ]:
# 비동기 스트리밍
print("[비동기 스트리밍]\n")

async def stream_response():
    async for chunk in llm.astream("JavaScript의 장점 3가지를 간단히 설명해줘"):
        print(chunk.content, end="", flush=True)
    print("\n\n[완료]")

await stream_response()

## 3. LangGraph 에이전트 스트리밍

ReAct 에이전트의 각 단계를 스트리밍으로 받습니다.

In [ ]:
from langchain_core.tools import tool
from langgraph.prebuilt import create_react_agent


@tool
def search(query: str) -> str:
    """Search for information on the web."""
    # 시뮬레이션
    return f"'{query}'에 대한 검색 결과: 관련 정보를 찾았습니다."


@tool
def calculate(expression: str) -> str:
    """Calculate a math expression."""
    try:
        result = eval(expression)
        return f"{expression} = {result}"
    except:
        return "계산 오류"


tools = [search, calculate]

# Tool calling 지원 모델 사용 (gemma3는 지원 안 함, qwen3 사용)
llm_for_agent = ChatOllama(
    model="qwen3:8b",  # tool calling 지원 모델
    base_url=OLLAMA_URL,
    temperature=0,
)

agent = create_react_agent(llm_for_agent, tools)
print("에이전트 생성 완료!")

In [ ]:
# stream 모드: 노드 단위 업데이트
print("[stream 모드 - 노드 단위]\n")

for event in agent.stream({"messages": [("human", "123 * 456을 계산해줘")]}):
    for node_name, node_output in event.items():
        print(f"=== {node_name} ===")
        if "messages" in node_output:
            for msg in node_output["messages"]:
                if hasattr(msg, "tool_calls") and msg.tool_calls:
                    for tc in msg.tool_calls:
                        print(f"  [Tool Call] {tc['name']}({tc['args']})")
                elif msg.content:
                    print(f"  [Content] {msg.content[:100]}..." if len(str(msg.content)) > 100 else f"  [Content] {msg.content}")
        print()

## 4. astream_events (상세 이벤트)

모든 내부 이벤트를 세밀하게 추적할 수 있습니다.

In [ ]:
print("[astream_events - 상세 이벤트]\n")

async def stream_events():
    async for event in agent.astream_events(
        {"messages": [("human", "100 + 200을 계산해줘")]},
        version="v2"
    ):
        kind = event["event"]
        
        if kind == "on_chat_model_start":
            print(f"🚀 [LLM 시작]")
        
        elif kind == "on_chat_model_stream":
            content = event["data"]["chunk"].content
            if content:
                print(content, end="", flush=True)
        
        elif kind == "on_chat_model_end":
            print(f"\n✅ [LLM 완료]")
        
        elif kind == "on_tool_start":
            print(f"\n🔧 [Tool 시작] {event['name']}")
            print(f"   입력: {event['data'].get('input', {})}")
        
        elif kind == "on_tool_end":
            print(f"🔧 [Tool 완료] {event['name']}")
            print(f"   결과: {event['data'].get('output', '')}")

await stream_events()

## 5. 토큰 단위 스트리밍 (LLM만)

In [ ]:
print("[토큰 단위 스트리밍]\n")
print("질문: 인공지능의 미래에 대해 설명해줘\n")
print("-" * 50)

token_count = 0

async def stream_tokens():
    global token_count
    async for chunk in llm.astream("인공지능의 미래에 대해 3문장으로 설명해줘"):
        token_count += 1
        print(chunk.content, end="", flush=True)
    
    print(f"\n\n----- 총 {token_count}개 청크 수신 -----")

await stream_tokens()

## 6. Socket.IO 서버 예제 코드

실제 웹소켓 서버를 구축할 때 사용할 수 있는 코드입니다.

In [ ]:
# Socket.IO 서버 코드 예시 (별도 파일로 저장하여 실행)

server_code = '''
import asyncio
import socketio
from aiohttp import web
from langchain_ollama import ChatOllama
from langchain_core.messages import HumanMessage

# Socket.IO 서버 설정
sio = socketio.AsyncServer(async_mode="aiohttp", cors_allowed_origins="*")
app = web.Application()
sio.attach(app)

# LLM 설정
llm = ChatOllama(
    model="gemma3:12b",
    base_url="http://localhost:11434",
    temperature=0.7,
)


@sio.event
async def connect(sid, environ):
    print(f"클라이언트 연결: {sid}")


@sio.event
async def disconnect(sid):
    print(f"클라이언트 연결 해제: {sid}")


@sio.event
async def chat(sid, data):
    """채팅 메시지 처리 - 스트리밍 응답"""
    message = data.get("message", "")
    print(f"[{sid}] 메시지: {message}")
    
    try:
        # 스트리밍 시작 알림
        await sio.emit("stream_start", to=sid)
        
        # 토큰 단위 스트리밍
        async for chunk in llm.astream(message):
            if chunk.content:
                await sio.emit("token", {"content": chunk.content}, to=sid)
        
        # 스트리밍 완료 알림
        await sio.emit("stream_end", to=sid)
    
    except Exception as e:
        await sio.emit("error", {"message": str(e)}, to=sid)


if __name__ == "__main__":
    print("Socket.IO 서버 시작: http://localhost:8000")
    web.run_app(app, host="0.0.0.0", port=8000)
'''

print("[Socket.IO 서버 코드]")
print(server_code)

In [ ]:
# 클라이언트 코드 예시

client_code = '''
import asyncio
import socketio

sio = socketio.AsyncClient()

@sio.event
async def connect():
    print("서버에 연결됨")

@sio.event
async def stream_start():
    print("\n[응답 시작]")

@sio.event
async def token(data):
    print(data["content"], end="", flush=True)

@sio.event
async def stream_end():
    print("\n[응답 완료]")

@sio.event
async def error(data):
    print(f"\n[에러] {data['message']}")

async def main():
    await sio.connect("http://localhost:8000")
    
    # 채팅 메시지 전송
    await sio.emit("chat", {"message": "Python에 대해 설명해줘"})
    
    # 응답 대기
    await asyncio.sleep(30)
    await sio.disconnect()

if __name__ == "__main__":
    asyncio.run(main())
'''

print("[Socket.IO 클라이언트 코드]")
print(client_code)

## 7. LangGraph + Socket.IO 통합 예제

In [ ]:
# LangGraph 에이전트 + Socket.IO 서버 코드

agent_server_code = '''
import asyncio
import socketio
from aiohttp import web
from langchain_ollama import ChatOllama
from langchain_core.tools import tool
from langgraph.prebuilt import create_react_agent

sio = socketio.AsyncServer(async_mode="aiohttp", cors_allowed_origins="*")
app = web.Application()
sio.attach(app)

# 도구 정의
@tool
def search(query: str) -> str:
    """Search the web for information."""
    return f"검색 결과: {query}에 대한 정보입니다."

@tool
def calculate(expression: str) -> str:
    """Calculate a math expression."""
    return str(eval(expression))

# 에이전트 설정
llm = ChatOllama(model="qwen3:8b", base_url="http://localhost:11434", temperature=0)
agent = create_react_agent(llm, [search, calculate])


@sio.event
async def chat(sid, data):
    message = data.get("message", "")
    
    try:
        async for event in agent.astream_events(
            {"messages": [("human", message)]},
            version="v2"
        ):
            kind = event["event"]
            
            if kind == "on_chat_model_stream":
                content = event["data"]["chunk"].content
                if content:
                    await sio.emit("token", {"content": content}, to=sid)
            
            elif kind == "on_tool_start":
                await sio.emit("tool_start", {
                    "name": event["name"],
                    "input": str(event["data"].get("input", {}))
                }, to=sid)
            
            elif kind == "on_tool_end":
                await sio.emit("tool_end", {
                    "name": event["name"],
                    "output": str(event["data"].get("output", ""))
                }, to=sid)
        
        await sio.emit("done", to=sid)
    
    except Exception as e:
        await sio.emit("error", {"message": str(e)}, to=sid)


if __name__ == "__main__":
    print("Agent + Socket.IO 서버: http://localhost:8000")
    web.run_app(app, host="0.0.0.0", port=8000)
'''

print("[LangGraph 에이전트 + Socket.IO 서버]")
print(agent_server_code)

## 8. 이벤트 타입 정리

| 이벤트 | 설명 | 데이터 |
|--------|------|--------|
| `on_chat_model_start` | LLM 호출 시작 | 모델 정보 |
| `on_chat_model_stream` | 토큰 생성 | `chunk.content` |
| `on_chat_model_end` | LLM 호출 완료 | 전체 응답 |
| `on_tool_start` | 도구 호출 시작 | 도구명, 입력값 |
| `on_tool_end` | 도구 호출 완료 | 도구명, 결과 |
| `on_chain_start` | 체인 시작 | - |
| `on_chain_end` | 체인 완료 | 최종 결과 |

## 9. 요약

### 스트리밍 방식 선택

```python
# 1. 단순 토큰 스트리밍
async for chunk in llm.astream("질문"):
    print(chunk.content)

# 2. 노드 단위 업데이트
for event in agent.stream(input):
    print(event)

# 3. 상세 이벤트 (권장)
async for event in agent.astream_events(input, version="v2"):
    if event["event"] == "on_chat_model_stream":
        print(event["data"]["chunk"].content)
```

### 웹소켓 연동 시
- `astream_events`로 모든 이벤트를 캡처
- 이벤트 타입별로 다른 Socket.IO 이벤트 emit
- 클라이언트에서 각 이벤트에 맞는 UI 업데이트